<a href="https://colab.research.google.com/github/Antony-6487/Vaccination_Sentiment_Analysis.ipynb/blob/main/Vaccination_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression
import numpy as np

Loading the dataset

In [60]:
Test=pd.read_csv('Test.csv')
Train=pd.read_csv('Train.csv')
SampleSubmission=pd.read_csv('SampleSubmission.csv')

Exploring the DATA

In [61]:
print(Train.head())

   tweet_id                                          safe_text  label  \
0  CL1KWCMY  Me &amp; The Big Homie meanboy3000 #MEANBOY #M...    0.0   
1  E3303EME  I'm 100% thinking of devoting my career to pro...    1.0   
2  M4IVFSMS  #whatcausesautism VACCINES, DO NOT VACCINATE Y...   -1.0   
3  1DR6ROZ4  I mean if they immunize my kid with something ...   -1.0   
4  J77ENIIE  Thanks to <user> Catch me performing at La Nui...    0.0   

   agreement  
0        1.0  
1        1.0  
2        1.0  
3        1.0  
4        1.0  


In [62]:
print(Train.shape, Test.shape)

(10001, 4) (5177, 2)


In [63]:
# Checkin sentiment distribution
print(Train['label'].value_counts())

label
 0.000000    4908
 1.000000    4053
-1.000000    1038
 0.666667       1
Name: count, dtype: int64


In [64]:
#checking rows with missing labels
print(Train.isnull().sum())

tweet_id     0
safe_text    0
label        1
agreement    2
dtype: int64


In [65]:
#dropping rows with missng labels
Train=Train.dropna(subset=['label'])

DATA CLEANING

In [66]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words=set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [68]:
def clean_text(text):
  if pd.isnull('text'):
    return''

In [69]:


basic_stopwords = {"the","a","an","and","or","in","of","on","for","with","at","by","to","is","are"}

def clean_text(text):
    if pd.isnull(text):
        return ""
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    text = ' '.join(word for word in text.split() if word not in basic_stopwords)
    return text





In [70]:
Train['clean_text'] = Train['safe_text'].apply(clean_text)
Test['clean_text'] = Test['safe_text'].apply(clean_text)

In [71]:

print(Train[['safe_text', 'clean_text']].head())

                                           safe_text  \
0  Me &amp; The Big Homie meanboy3000 #MEANBOY #M...   
1  I'm 100% thinking of devoting my career to pro...   
2  #whatcausesautism VACCINES, DO NOT VACCINATE Y...   
3  I mean if they immunize my kid with something ...   
4  Thanks to <user> Catch me performing at La Nui...   

                                          clean_text  
0  me amp big homie meanboy meanboy mb mbs mmr st...  
1  im thinking devoting my career proving autism ...  
2  whatcausesautism vaccines do not vaccinate you...  
3  i mean if they immunize my kid something that ...  
4  thanks user catch me performing la nuit nyc st...  


In [72]:
Train['safe_text'] = Train['safe_text'].fillna('')
Test['safe_text'] = Test['safe_text'].fillna('')


VECTORIZATION

In [79]:
#converting string to numerical data

vectorizer = TfidfVectorizer(
    max_features=10000,
    ngram_range=(1,2),     # unigrams + bigrams
    stop_words='english'
)

X_train = vectorizer.fit_transform(Train['safe_text'])
X_test = vectorizer.transform(Test['safe_text'])


MODEL TRAINING

In [96]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
import numpy as np

model = Ridge(alpha=1.0)
model.fit(X_train, y_train)

y_pred = model.predict(X_train)
rmse = np.sqrt(mean_squared_error(y_train, y_pred))
print("Training RMSE:", rmse)


Training RMSE: 0.39839858588543225


In [97]:
y_train = Train['label']

In [98]:
print(y_train.describe())


count    10000.000000
mean         0.301567
std          0.646718
min         -1.000000
25%          0.000000
50%          0.000000
75%          1.000000
max          1.000000
Name: label, dtype: float64


In [94]:
submission = pd.DataFrame({
    'ID': Test['tweet_id'],
    'target': y_pred
})

submission.to_csv('submission.csv', index=False)
print("submission.csv created successfully!")


submission.csv created successfully!


In [99]:
from google.colab import files
files.download('submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>